### Mercari Price 
The files consist of a list of product listings. These files are tab-delimited.

Fields:
- train_id or test_id - the id of the listing

- name - the title of the listing. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid  leakage. These removed prices are represented as [rm]

- item_condition_id - the condition of the items provided by the seller

- category_name - category of the listing

- brand_name

- price - the price that the item was sold for. This is the target variable that you will predict. The unit is USD. This column doesn't exist in test.tsv since that is what you will predict.

- shipping - 1 if shipping fee is paid by seller and 0 by buyer

- item_description - the full description of the item. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm]

## Setup

In [ ]:
! pip install pydot graphviz emoji transformers

import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow.keras.backend as K


from tqdm._tqdm_notebook import tqdm_notebook

import os
import itertools

import matplotlib.pyplot as plt
import itertools
from collections import Counter
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import sklearn

from tensorflow.keras.layers import Dense, Input, Embedding, Concatenate, Flatten, Dropout, LSTM, GlobalMaxPool1D, GRU, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

from nltk.corpus import stopwords
import string

import pickle

from tensorflow.keras.preprocessing.text import text_to_word_sequence 
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import TweetTokenizer

import emoji
import os

import transformers
from transformers import DistilBertTokenizerFast, TFDistilBertModel, pipeline

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


stop_words = set(stopwords.words('english'))
stop_words.remove("no")

tqdm_notebook.pandas()

In [ ]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return K.sqrt(msle(y_true, y_pred))

In [ ]:
# seed
random_seed = 1000
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

## Colab

In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython())

# check if in colab
if RunningInCOLAB and not os.path.isdir('/content/gdrive'):
    print("Running in colab")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    colab_root = '/content/drive'
      
if RunningInCOLAB:
    root_dir = "/content/gdrive/My Drive/"
    base_dir = root_dir + 'project-mercari-price/'
    if not os.path.isdir(base_dir):
        os.mkdir(base_dir)
else:
    root_dir= os.getcwd()
    base_dir = root_dir

os.chdir(base_dir)

os.getcwd()

## Dataset download

In [ ]:
dataset_downloaded_path = os.path.join(base_dir, "dataset_downloaded.ignore")
dataset_downloaded = os.path.isfile(dataset_downloaded_path)
dataset_downloaded

if not dataset_downloaded:
  # install kaggle to download dataset
  ! pip install kaggle python-dotenv

# set to True if you want to save kaggle credentials into a .env file
persist_credentials = False

if not dataset_downloaded:
  # create .env file containing KAGGLE_USER and KAGGLE_KEY
    kaggle_env = os.path.join(base_dir, '.env')
    if not os.path.isfile(kaggle_env):
        with open(kaggle_env, 'w') as envfile:
            kaggle_user = input("Insert kaggle username")
            kaggle_key = input("Insert kaggle key; generate one from kaggle account")
        if persist_credentials:
            envfile.write(f"""
            KAGGLE_USERNAME={kaggle_user}
            KAGGLE_KEY={kaggle_key}
            """)

        # set env vars
        os.environ["KAGGLE_USERNAME"] = kaggle_user
        os.environ["KAGGLE_KEY"] = kaggle_key

        del kaggle_user
        del kaggle_key

if not dataset_downloaded:
  # loading env vars if .env file exists
    if os.path.isfile(kaggle_env):
        from dotenv import load_dotenv
        load_dotenv(dotenv_path=kaggle_env)
    print(os.environ.get("KAGGLE_USERNAME"))

if not dataset_downloaded:
    # download and extract dataset
    ! kaggle competitions download -c mercari-price-suggestion-challenge

    # create file so that we know we already downloaded
    with open(dataset_downloaded_path, 'w') as dd_file:
        dataset_downloaded = True
        dd_file.write("")

    print('cwd: ', os.getcwd())
    
    os.listdir()

if not dataset_downloaded:
    ! 7z x train.tsv.7z
    ! 7z x test.tsv.7z

os.listdir()

## Load dataset

In [ ]:
dtypes={
    'name': 'string',
    'item_condition_id': 'int32',
    'category_name': 'string',
    'brand_name': 'string',
    'price': 'float',
    'shipping': 'int32',
    'item_description': 'string'
}
data = pd.read_csv("train.tsv", sep='\t', dtype=dtypes)
data = data.drop(columns=["train_id"])
print(data.dtypes)
print(data.shape)
data

In [ ]:
dtypes={
    'name': 'string',
    'item_condition_id': 'int32',
    'category_name': 'string',
    'brand_name': 'string',
    'price': 'float',
    'shipping': 'int32',
    'item_description': 'string'
}
test = pd.read_csv("test.tsv", sep='\t', dtype=dtypes)
test = test.drop(columns=["test_id"])
print(test.dtypes)
print(test.shape)
test

In [ ]:
for column in data.columns:
    print("number of null value in {} : {}".format(column,data[column].isnull().sum()))

In [ ]:
for column in test.columns:
    print("number of null value in {} : {}".format(column,test[column].isnull().sum()))

In [ ]:
data["price"].describe()

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="price",)
fig.show()

In [ ]:
data["shipping"].value_counts()

In [ ]:
data["shipping"].value_counts()[0]/len(data["shipping"])*100

In [ ]:
data[data["shipping"]==0]["price"].mean()

In [ ]:
data[data["shipping"]==1]["price"].mean()

In [ ]:
fig = px.histogram(data, x="price", color="shipping")
fig.update_layout(xaxis_type="log", yaxis_type="log")

fig.show()

In [ ]:
data["category_name"].value_counts()

In [ ]:
data["item_condition_id"].value_counts()

In [ ]:
data[data["item_condition_id"]==1]["price"].mean()

In [ ]:
data[data["item_condition_id"]==2]["price"].mean()

In [ ]:
data[data["item_condition_id"]==3]["price"].mean()

In [ ]:
data[data["item_condition_id"]==4]["price"].mean()

In [ ]:
data[data["item_condition_id"]==5]["price"].mean()

In [ ]:
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

In [ ]:
data['general_cat'], data['subcat_1'], data['subcat_2'] = \
zip(*data['category_name'].apply(lambda x: split_cat(x)))
data.head()

In [ ]:
data["brand_name"].value_counts()

In [ ]:
len(data[data["item_description"].str.lower() == "no description yet"])

In [ ]:
data["len_desc"]=data['item_description'].str.len()

In [ ]:
data[["price","len_desc"]].corr(method='pearson')

In [ ]:
uno=data[data["price"]>=100]

In [ ]:
due=data[(data["price"]>50) & (data["price"]<100)]

In [ ]:
tre=data[(data["price"]>30) & (data["price"]<=50)]

In [ ]:
quattro=data[data["price"]<=30]

In [ ]:
def flat_list(l):
    return  [item for sublist in l for item in sublist]

In [ ]:
def word_Cloud(sentences):
    flat_sentences = flat_list(sentences)
    print(flat)
    counter = Counter(flat_sentences)
    cdict = dict(counter.most_common())

    wcloud = wordcloud.WordCloud(background_color="white").generate_from_frequencies(cdict)
    plt.figure(figsize = (10, 8), facecolor = None) 
    plt.imshow(wcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
data["item_description"] = data["item_description"].fillna(value="NA")
data["brand_name"] = data["brand_name"].fillna(value="NA")
data["category_name"] = data["category_name"].fillna(value="NA")
# see warnings -> inplace?
data.shape

In [ ]:
uno["bigrams_description"]=(uno["item_description"].apply(text_to_word_sequence)).progress_apply(lambda row: list(nltk.ngrams(row, 2)))
uno["bigrams_description"]= uno["bigrams_description"].progress_apply(lambda tupla: [' '.join(elem) for elem in tupla])
word_Cloud(uno["bigrams_description"])

In [ ]:
due["bigrams_description"]=(due["item_description"].apply(text_to_word_sequence)).progress_apply(lambda row: list(nltk.ngrams(row, 2)))
due["bigrams_description"]= due["bigrams_description"].progress_apply(lambda tupla: [' '.join(elem) for elem in tupla])
word_Cloud(due["bigrams_description"])

In [ ]:
tre["bigrams_description"]=(tre["item_description"].apply(text_to_word_sequence)).progress_apply(lambda row: list(nltk.ngrams(row, 2)))
tre["bigrams_description"]= tre["bigrams_description"].progress_apply(lambda tupla: [' '.join(elem) for elem in tupla])
word_Cloud(tre["bigrams_description"])

In [ ]:
quattro["bigrams_description"]=(quattro["item_description"].apply(text_to_word_sequence)).progress_apply(lambda row: list(nltk.ngrams(row, 2)))
quattro["bigrams_description"]= quattro["bigrams_description"].progress_apply(lambda tupla: [' '.join(elem) for elem in tupla])
word_Cloud(quattro["bigrams_description"])

# Data cleaning

Handle missing values and wrong prices.

Prices should be in the range [5,2000]

https://www.mercari.com/us/help_center/article/69

In [ ]:
len(data[(data["price"]<5) | (data["price"]>2000)])

In [ ]:
data=data[(data["price"]>=5) & (data["price"]<=2000)]

In [ ]:
test["brand_name"] = test["brand_name"].fillna(value="NA")
test["category_name"] = test["category_name"].fillna(value="NA")
# see warnings -> inplace?
test.shape

In [ ]:
data

# Preprocessing

In [ ]:
data["item_description"]=data["item_description"].str.lower()
data["name"]=data["name"].str.lower()
data.head()

In [ ]:
data["item_description"]=data["item_description"].replace("no description yet", "NA")

## Text cleanup

In [ ]:
tweetTokenizer = TweetTokenizer()

def list_to_str(l):
   return ' '.join([str(elem) for elem in l])
   
def textCleanup(df, flag=True):
  df=df.to_frame(name="str")
  #df["clean"] = df["str"].progress_apply(text_to_word_sequence)   # 20 secondi
  global tweetTokenizer
  df["clean"] = df["str"].progress_apply(tweetTokenizer.tokenize) # 2 minutes but correctly handles emojis

  # punct and stop words
  

  lemmatizer = WordNetLemmatizer() 
  
  df["clean"] = df["clean"].progress_apply(lambda sentence : [lemmatizer.lemmatize(word) for word in sentence if word not in stop_words]) # 10 secondi
  df["clean"] = df["clean"].progress_apply(lambda sentence:
                                           [w for w in sentence if w
                                              not in string.punctuation
                                              and w not in stop_words and (len(w)>1 or w.isdigit()) and w not in emoji.UNICODE_EMOJI]) # 18 s
  plot_common_tokens(df["clean"], "Most Common Tokens without StopWords", n=20)
  if flag:
    df["clean"] = df["clean"].progress_apply(list_to_str) # 6 secondi
  return df["clean"]

In [ ]:
def preprocessData(data):
  print('description clean up')
  data["item_description_clean"] = textCleanup(data["item_description"]) 

  print('name clean up')
  data["name_clean"] = textCleanup(data["name"])
  
  return data

In [ ]:
def flat_list(l):
    return  [item for sublist in l for item in sublist]

In [ ]:
def plot_common_tokens(tokens, title, n=20):
    sentences = (list(itertools.chain(tokens)))
    flat_sentences = flat_list(sentences)
    counts = Counter(flat_sentences)
    #print(counts.most_common(30))
    common_words = [word[0] for word in counts.most_common(n)]
    common_counts = [word[1] for word in counts.most_common(n)]
    fig = plt.figure(figsize=(18,6))
    sns.barplot(x=common_words, y=common_counts)
    plt.title(title)
    plt.show()

In [ ]:
data = preprocessData(data)
data.head()

In [ ]:
test = preprocessData(test)
test.head()

## Categories split

Most category_name (~99.7%) contains 3 subcategories or less.

We consider the first 3 subcategories

In [ ]:
data["category_name"].apply(lambda x: x.count('/')+1).value_counts().sort_index() / data.shape[0]

In [ ]:
def split_cat(text):
  """
  function to split category into 3 subcategories

  e.g:
  Men/Tops/T-shirts -> ["Men", "Tops", "T-shirts"]

  in case of more than 3 subcategories (~0.3% of the dataset)
  the last categories will be bounded toghether e.g.
  
  Men/Tops/T-shirts/RedTshirts/FlameTshirts ->
  -> ["Men", "Tops", "T-shirts/RedTshirts/FlameTshirts"]

  in case of less than 3 if fills with 'NA'
  """
  r = text.split("/", 2)
  while len(r) < 3:
    r = r + ['NA']
  return r

In [ ]:
data['cat1'], data['cat2'], data['cat3'] = zip(*data['category_name'].progress_apply(lambda x: split_cat(x)))
data.head()

# Test preprocessing

## Text cleanup

In [ ]:
test["item_description"]=test["item_description"].str.lower()
test["name"]=test["name"].str.lower()
test.head()

In [ ]:
test = preprocessData(test)
test.head()

## Categories split

In [ ]:
test['cat1'], test['cat2'], test['cat3'] = zip(*test['category_name'].progress_apply(lambda x: split_cat(x)))
test.head()

# Load preprocessed

In [ ]:
fname = 'train_preprocess.npy'
if 'data' in globals():
  # update preprocessed data
  print("saving preprocess train data")
  data.to_pickle(fname)
else:
  print("loading existing preprocess train data")
  data = pd.read_pickle(fname)

In [ ]:
fname = 'test_preprocess.npy'

if 'test' in globals():
  # update preprocess test data
  print("saving preprocess test data")
  test.to_pickle(fname)
else:
  print("loading existing preprocess test data")
  test = pd.read_pickle(fname)

# Train Validation split

In [ ]:
train, validation = train_test_split(data, test_size=0.2, random_state=random_seed)

# Categorical Encoding

In [ ]:
# TODO ensure it is correct or use a well tested alternative like sklearn (found problems with dimensions)
class LabelEncoder:
  """
  Simple single dimension label encoder class able to handle
  unknown values 
  """
  def __init__(self, unknown = 0, invUnknown = 'unknown'):
    self.leDict = {}
    self.invDict = {}
    self.unknown = unknown
    self.invUnknown = invUnknown

  def fit(self, data):
    vci = pd.value_counts(data).index
    self.leDict = dict(zip(vci, range(1, len(vci)+1)))
    self.invDict = dict(zip(range(1, len(vci)+1), vci))

  def transform1(self, item):
    return self.leDict.get(item, self.unknown)

  def transform(self, data):
    return data.apply(lambda item: self.transform1(item))

  def inverse_transform1(self, item):
    return self.invDict.get(item, self.invUnknown)

  def inverse_transform(self, data):
    return data.apply(lambda item: self.inverse_transform1(item))

In [ ]:
cat_le = LabelEncoder()
cat_le.fit(np.hstack([train["cat1"], train['cat2'], train['cat3']]))

In [ ]:
train["cat1_l"] = cat_le.transform(train["cat1"])
train["cat2_l"] = cat_le.transform(train["cat2"])
train["cat3_l"] = cat_le.transform(train["cat3"])

In [ ]:
brand_le = LabelEncoder()
brand_le.fit(train["brand_name"])

In [ ]:
train["brand_name_l"] = brand_le.transform(train["brand_name"])

## Validation

In [ ]:
validation["cat1_l"] = cat_le.transform(validation["cat1"])
validation["cat2_l"] = cat_le.transform(validation["cat2"])
validation["cat3_l"] = cat_le.transform(validation["cat3"])

validation["brand_name_l"] = brand_le.transform(validation["brand_name"])

## Test

In [ ]:
test["cat1_l"] = cat_le.transform(test["cat1"])
test["cat2_l"] = cat_le.transform(test["cat2"])
test["cat3_l"] = cat_le.transform(test["cat3"])

test["brand_name_l"] = brand_le.transform(test["brand_name"])

# Categorical only

In [ ]:
# avoid to mess up original train and validation
train_cat = train
validation_cat = validation

In [ ]:
def getModelCat():
    inputA = Input(shape=(6,))

    x = Dense(32, activation='relu')(inputA)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=inputA, outputs=x)
    
    return model

In [ ]:
model_cat = getModelCat()
model_cat.summary()

In [ ]:
plot_model(model_cat)

In [ ]:
model_cat.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
inputA_train_cat = train_cat[["item_condition_id", "cat1_l", "cat2_l", "cat3_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
inputA_validation_cat = validation_cat[["item_condition_id", "cat1_l", "cat2_l", "cat3_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
y_train_cat = train_cat["price"].values

In [ ]:
y_validation_cat = validation_cat["price"].values

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

history_cat = model_cat.fit(x=inputA_train_cat, y=y_train_cat,
                    epochs=10,
                    verbose=True,
                    validation_data=(inputA_validation_cat, y_validation_cat),
                    callbacks=[callback],
                    batch_size=512)


10/10 2261/2261 [==============================] - 5s 2ms/step - loss: 0.6769 - mse: 1529.0349 - mae: 15.5510 - root_mean_squared_error: 39.1006 - mean_squared_logarithmic_error: 0.4589 - root_mean_squared_logarithmic_error: 0.6769 - val_loss: 0.6703 - val_mse: 1456.1018 - val_mae: 15.3929 - val_root_mean_squared_error: 38.1589 - val_mean_squared_logarithmic_error: 0.4501 - val_root_mean_squared_logarithmic_error: 0.6705

CHANGED. this was the one with single category


In [ ]:
histDf_cat = pd.DataFrame(history_cat.history)
histDf_cat.tail()

# Keras Embedding

In [ ]:
name_length_max=10
desc_length_max=75

In [ ]:
train_keras = train
validation_keras = validation

In [ ]:
# tokenize with keras; it also does some encoding
def tokenizeData(df, description, name, tokenizer= None, texts= None):
  if tokenizer == None:
    tokenizer = Tokenizer()

    print("fit tokenizer")
    tokenizer.fit_on_texts(texts)
  
  print('tokenize description')
  df["item_description_t"]=tokenizer.texts_to_sequences(df[description])

  print('tokenize name')
  df["name_t"]=tokenizer.texts_to_sequences(df[name])
  return df, tokenizer

In [ ]:
texts = np.hstack([train_keras["item_description_clean"], train["name_clean"]])

In [ ]:
train_keras, tokenizer = tokenizeData(
    train_keras ,"item_description_clean" ,"name_clean", None, texts)

In [ ]:
validation_keras, _ = tokenizeData(
    validation_keras ,"item_description_clean" ,"name_clean", tokenizer, None)

In [ ]:
vocab_size= len(tokenizer.word_index)+1
print(vocab_size)
# clean con trattamento emoji 193300
# clean 246054
# no clean 255431 (considerando tutto anche punteggiatura)

In [ ]:
train_keras[["item_description_t","name_t"]]

In [ ]:
inputDesc_train_keras = pad_sequences(train_keras["item_description_t"],
                                                  padding='post', maxlen=desc_length_max)

In [ ]:
inputDesc_validation_keras = pad_sequences(validation_keras["item_description_t"],
                                                  padding='post', maxlen=desc_length_max)

In [ ]:
inputName_train_keras = pad_sequences(train_keras["name_t"], padding='post', maxlen=name_length_max)

In [ ]:
inputName_validation_keras = pad_sequences(validation_keras["name_t"], padding='post', maxlen=name_length_max)

In [ ]:
y_train_keras = train_keras["price"]
y_train_keras

In [ ]:
y_validation_keras = validation_keras["price"]
y_validation_keras

In [ ]:
inputA_train_keras = train_keras[["item_condition_id", "cat1_l", "cat2_l", "cat3_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
inputA_validation_keras = validation_keras[["item_condition_id", "cat1_l", "cat2_l", "cat3_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
name_vocab_size= vocab_size
desc_vocab_size= vocab_size
def getModelKeras(name_length_max, desc_length_max, name_vocab_size, desc_vocab_size):
    inputA = Input(shape=(6,))
    #Ad = Dense(4, activation='relu')(inputA)
    Ad = inputA
    
    inputName = Input(shape=(name_length_max,))
    Ne = Embedding(input_dim=name_vocab_size, output_dim=50, input_length=name_length_max)(inputName)
    #Ne = Embedding(
    #    num_tokens,
    #    embedding_dim,
    #    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    #    trainable=False,
    #)(inputName)
    Nd = Bidirectional(LSTM(12, return_sequences=True, dropout=0.2))(Ne)
    Nd =  GlobalMaxPool1D()(Nd)

    #Nd = Nf
    
    inputDesc = Input(shape=(desc_length_max,))
    De = Embedding(input_dim=desc_vocab_size, output_dim=50, input_length=desc_length_max)(inputDesc)
    #De = Embedding(
    #    num_tokens,
    #    embedding_dim,
    #    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    #    trainable=False,
    #)(inputDesc)
    Dd = Bidirectional(LSTM(16, return_sequences=True, dropout=0.2))(De)
    Dd = Bidirectional(LSTM(8, return_sequences=True, dropout=0.2))(Dd)
    
    Dd =  GlobalMaxPool1D()(Dd)
    #Dd = Df
    
    concat = Concatenate()([Ad, Nd, Dd])

    x = Dropout(0.2)(concat)    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputA, inputName, inputDesc], outputs=x)
    
    return model
    

In [ ]:
model_keras = getModelKeras(name_length_max, desc_length_max, name_vocab_size, desc_vocab_size)
model_keras.summary()

In [ ]:
plot_model(model_keras)

In [ ]:
model_keras.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

https://machinelearningmastery.com/clean-text-machine-learning-python/ ultima sezione

In [ ]:
callbacks = [
             tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss',
                  patience=3,
                  restore_best_weights=True
                  )
             ]
history_keras = model_keras.fit(x=[inputA_train_keras,inputName_train_keras, inputDesc_train_keras], y=y_train_keras,
                    epochs=10,
                    verbose=True,
                    validation_data=([inputA_validation_keras ,inputName_validation_keras, inputDesc_validation_keras], y_validation_keras),
                    callbacks=[callback],
                    batch_size=512)
# senza pulizia  loss: 0.4100  e val_root_mean_squared_logarithmic_error: 0.4543
# con pulizia leggera (lower)  val_root_mean_squared_logarithmic_error: 0.455 e loss: 0.4
# con pulizia val_root_mean_squared_logarithmic_error: 0.4643 e loss: 0.4269
# con lunghezza embedding 50 e senza pulizia val_root_mean_squared_logarithmic_error: 0.4556 e loss: 0.3675 con 10 epoche 

In [ ]:
histDf_keras = pd.DataFrame(history_keras.history)
histDf_keras.tail()

In [ ]:
histDf_keras.plot(y=["loss", "val_loss"])

### Test

In [ ]:
test_keras, _ = tokenizeData(test ,"item_description", "name", tokenizer)

In [ ]:
test_keras=test[["item_condition_id","shipping","category_name_l","brand_name_l", "item_description_t", "name_t"]]
test_keras

In [ ]:
inputA_test_keras = test_keras[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')
inputName_test_keras = pad_sequences(test_keras["name_t"], padding='post', maxlen=name_length_max)
inputDesc_test_keras = pad_sequences(test_keras["item_description_t"], padding='post', maxlen=desc_length_max)
pred_keras = model_keras.predict([inputA_test_keras, inputName_test_keras, inputDesc_test_keras])
pred_keras

In [ ]:
import math

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
validation_keras

In [ ]:
pred_keras=pred_keras.round()

In [ ]:
pred_v_keras = model.predict([inputA_validation_keras, inputName_validation_keras, inputDesc_validation_keras])
pred_v_keras = pred_v_keras.round()

In [ ]:
len(pred_v_keras)

In [ ]:
len(y_validation_keras)

In [ ]:
root_mean_squared_logarithmic_error(y_validation_keras, pred_v_keras)

# Glove pretrained

In [ ]:
train_glove = train
validation_glove = validation

https://nlp.stanford.edu/projects/glove/

In [ ]:
#archive_url = 'http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip'
#archive_url = "https://github.com/facebookresearch/fastText/archive/v0.9.2.zip"
#archive_url = "https://www.cs.uic.edu/~hxu/ele_review_qa_300d.tar.gz"
archive_url = 'http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.840B.300d.zip'
archive_name = 'glove.840B.300d.zip'
remove_archive = False
embedding_fname = 'glove.840B.300d.txt'

In [ ]:
if not os.path.isfile(archive_name) and not os.path.isfile(embedding_fname):
  ! wget {archive_url} -O {archive_name}

In [ ]:
if not os.path.isfile(embedding_fname):
  ! unzip {archive_name} {embedding_fname}

# remove archive if already extracted
if remove_archive and os.path.isfile(embedding_fname):
  os.remove(archive_name)

Needs a word encoding index (e.g. keras one)

TODO: creare un word index da qui se non c'è

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))

In [ ]:
path_to_glove_file = os.path.join(
    base_dir, embedding_fname
)

'''embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))'''

In [ ]:
path_to_glove_file

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index (padding)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
              i = word_index[word]
              try: 
                embedding_matrix[i] = np.array(vector, dtype=np.float32)[:embedding_dim]
              except:
                print(word)
                print(vector)
                print("_______________________")

    return embedding_matrix

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index (padding)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
              i = word_index[word]
              try: 
                embedding_matrix[i] = np.array(vector, dtype=np.float32)[:embedding_dim]
                
  
              except:

                vector=vector[1:embedding_dim+1]
                embedding_matrix[i] = np.array(vector, dtype=np.float32)


    return embedding_matrix

In [ ]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix(path_to_glove_file, tokenizer.word_index, embedding_dim)

In [ ]:
embedding_matrix[10]

In [ ]:
embedding_matrix[107098]

In [ ]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size
# senza pulizia viene coperto il 28% del vocabolario
# con pulizia il 26% senza emoji
# con pulizia con emoji 34%
# 0.3779 senza pulizia con glove più grande (glove.840B.300d)
# 0.467 con pulizia con glove più grande (glove.840B.300d)

In [ ]:
'''#glove pretrained embedding
num_tokens = len(word_index) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))'''

**IMPORTANTISSIMO: SCEGLIERE CHE LUNGHEZZA USARE PER L'ECONDING DELLE PAROLE DELLE DESCRIZIONI E DEI NOMI, CIOE' SE 8 VALORI ENTRAMBI O MANTENERE 8 E 16 VALORI**

In [ ]:
name_vocab_size= vocab_size
desc_vocab_size= vocab_size
def getModelGlove(name_length_max, desc_length_max, name_vocab_size, desc_vocab_size, embedding_matrix=None):
    inputA = Input(shape=(4,))
    Ad = inputA
    
    inputName = Input(shape=(name_length_max,))
    if embedding_matrix.size == 0:
      Ne = Embedding(input_dim=name_vocab_size, output_dim=50, weights=[embedding_matrix], input_length=name_length_max, trainable=False)(inputName)
    else: 
      Ne = Embedding(input_dim=name_vocab_size, output_dim=50, input_length=name_length_max)(inputName)

    Nd = Bidirectional(LSTM(12, return_sequences=True, dropout=0.2))(Ne)
    Nd =  GlobalMaxPool1D()(Nd)
    
    inputDesc = Input(shape=(desc_length_max,))
    if embedding_matrix.size == 0:
      De = Embedding(input_dim=desc_vocab_size, output_dim=50, weights=[embedding_matrix], input_length=desc_length_max, trainable=False)(inputDesc)
    else:
      De = Embedding(input_dim=desc_vocab_size, output_dim=50, input_length=desc_length_max)(inputDesc)


    Dd = Bidirectional(LSTM(16, return_sequences=True, dropout=0.2))(De)
    Dd = Bidirectional(LSTM(8, return_sequences=True, dropout=0.2))(Dd)
    
    Dd =  GlobalMaxPool1D()(Dd)
    
    concat = Concatenate()([Ad, Nd, Dd])

    x = Dropout(0.2)(concat)    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputA, inputName, inputDesc], outputs=x)
    
    return model
    

In [ ]:
model_glove= getModelGlove(name_length_max, desc_length_max, name_vocab_size, desc_vocab_size, embedding_matrix)
model_glove.summary()

In [ ]:
plot_model(model_glove)

In [ ]:
model_glove.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

TODO input glove and not keras

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

history_glove = model_glove.fit(x=[inputA_train_keras,inputName_train_keras, inputDesc_train_keras], y=y_train_keras,
                    epochs=10,
                    verbose=True,
                    validation_data=([inputA_validation_keras ,inputName_validation_keras, inputDesc_validation_keras], y_validation_keras),
                    callbacks=[callback],
                    batch_size=512)
# glove 6M 
# con 10 epoche e senza pulizia: loss: 0.5564 e val_root_mean_squared_logarithmic_error: 0.5392
# con 10 epoche e pulizia emoji etc: loss: 0.5553 e val_root_mean_squared_logarithmic_error: 0.5312

# glove.840B.300d
# con 10 epoche senza pulizia: loss: 0.3678  e val_root_mean_squared_logarithmic_error: 0.4537
# con 10 epoce con pulizia: loss: 0.3817 e val_root_mean_squared_logarithmic_error 0.4604

# BoW

In [ ]:
train_cvec = train
validation_cvec = validation

In [ ]:
categorical_train_cvec= train_cvec[["item_condition_id","shipping", "cat1_l", "cat2_l", "cat3_l", "brand_name_l"]]
print("Train:")
print(categorical_train_cvec.head())
print("\nValidation:")
categorical_validation_cvec= validation_cvec[["item_condition_id","shipping","cat1_l", "cat2_l", "cat3_l","brand_name_l"]]
print(categorical_validation_cvec.head())

In [ ]:
y_train_cvec=train_cvec["price"].values
y_validation_cvec=validation_cvec["price"].values

## Count Vectorizer

In [ ]:
vectorizer_desc = CountVectorizer()
vectorizer_desc.fit(train_cvec["item_description"].values)

In [ ]:
len(vectorizer_desc.vocabulary_)

In [ ]:
description_train_cvec = vectorizer_desc.transform(train_cvec["item_description"].values)

In [ ]:
description_validation_cvec = vectorizer_desc.transform(validation_cvec["item_description"].values)

In [ ]:
type(description_validation_cvec)

In [ ]:
vectorizer_name = CountVectorizer()
vectorizer_name.fit(train_cvec["name"].values)

In [ ]:
len(vectorizer_name.vocabulary_)

In [ ]:
name_train_cvec = vectorizer_name.transform(train_cvec["name"].values)
name_validation_cvec = vectorizer_name.transform(validation_cvec["name"].values)

In [ ]:
type(name_train_cvec)

In [ ]:
def getModel_bow(categorical_train, name_train, description_train):
    inputA = Input(categorical_train.shape[1])
    inputName = Input(name_train.shape[1])
    inputDesc = Input(description_train.shape[1])
    concat = Concatenate()([inputName, inputDesc, inputA])

    x = Dropout(0.1)(concat)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputName ,inputDesc, inputA], outputs=x)

    return model

In [ ]:
model_cvec = getModel_bow(categorical_train_cvec, name_train_cvec, description_train_cvec)
model_cvec.summary()

In [ ]:
plot_model(model_cvec)

In [ ]:
model_cvec.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
history_cvec = model_cvec.fit(x=[name_train_cvec, description_train_cvec, categorical_train_cvec.values],
                    y=y_train_cvec,
                    epochs=5,
                    verbose=True,
                    validation_data=(
                        [name_validation_cvec, description_validation_cvec, categorical_validation_cvec.values]
                        , y_validation_cvec),
                    batch_size=512)

BOW con pulizia:
loss: 0.4549 - mse: 877.1694 - mae: 10.7754 - root_mean_squared_error: 29.6144 - mean_squared_logarithmic_error: 0.2073 - root_mean_squared_logarithmic_error: 0.4549 - val_loss: 0.4572 - val_mse: 823.4496 - val_mae: 10.7306 - val_root_mean_squared_error: 28.6958 - val_mean_squared_logarithmic_error: 0.2095 - val_root_mean_squared_logarithmic_error: 0.4573

BOW senza pulizia words:
loss: 0.4537 - mse: 892.7444 - mae: 10.7434 - root_mean_squared_error: 29.8743 - mean_squared_logarithmic_error: 0.2063 - root_mean_squared_logarithmic_error: 0.4537 - val_loss: 0.4554 - val_mse: 848.4371 - val_mae: 10.7117 - val_root_mean_squared_error: 29.1279 - val_mean_squared_logarithmic_error: 0.2079 - val_root_mean_squared_logarithmic_error: 0.4555

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    rmsle = history.history['root_mean_squared_logarithmic_error']
    val_rmsle = history.history['val_root_mean_squared_logarithmic_error']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(rmsle) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, rmsle, 'b', label='Training rmsle')
    plt.plot(x, val_rmsle, 'r', label='Validation rmsle')
    plt.title('Training and validation root_mean_squared_logarithmic_error')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

## TF-IDF

In [ ]:
data["item_description"] = data["item_description"]
data["name"] = data["name"]

In [ ]:
tfidf_vectorizer_desc = TfidfVectorizer()
tfidf_vectorizer_desc.fit(train_cvec["item_description"].values)

In [ ]:
len(tfidf_vectorizer_desc.vocabulary_)

In [ ]:
tfidf_description_train = tfidf_vectorizer_desc.transform(train_cvec["item_description"].values)
tfidf_description_validation = tfidf_vectorizer_desc.transform(validation_cvec["item_description"].values)

In [ ]:
tfidf_vectorizer_name = TfidfVectorizer()
tfidf_vectorizer_name.fit(train_cvec["name"])

In [ ]:
len(tfidf_vectorizer_name.vocabulary_)

In [ ]:
tfidf_name_train = tfidf_vectorizer_name.transform(train_cvec["name"])
tfidf_name_validation = tfidf_vectorizer_name.transform(validation_cvec["name"])

In [ ]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [ ]:
type(tfidf_name_train) 

In [ ]:
#https://stackoverflow.com/questions/61961042/indices201-0-8-is-out-of-order-many-sparse-ops-require-sorted-indices-use

In [ ]:
import scipy
new_categorical_train_cvec=scipy.sparse.csr_matrix(categorical_train_cvec.values)
new_categorical_validation_cvec=scipy.sparse.csr_matrix(categorical_validation_cvec.values)

In [ ]:
new_tfidf_categorical_validation=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(new_categorical_validation_cvec))
new_tfidf_categorical_train=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(new_categorical_train_cvec))

In [ ]:
new_tfidf_name_validation=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(tfidf_name_validation))
new_tfidf_name_train=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(tfidf_name_train))

In [ ]:
type(new_tfidf_name_validation)

In [ ]:
type(tfidf_name_validation)

In [ ]:
new_tfidf_description_validation=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(tfidf_description_validation))
new_tfidf_description_train=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(tfidf_description_train))

In [ ]:
'''new_y_train_cvec=scipy.sparse.csr_matrix(y_train_cvec)
new_y_validation_cvec=scipy.sparse.csr_matrix(y_validation_cvec)'''

In [ ]:
'''new_tfidf_y_validation=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(new_y_validation_cvec))
new_tfidf_y_train=tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(new_y_train_cvec))'''

In [ ]:
'''type(tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(new_y_train_cvec)))'''

In [ ]:
def getModel_bow(name_train, description_train, categorical_train):
    inputA = Input(categorical_train.shape[1])
    inputName = Input(name_train.shape[1])
    inputDesc = Input(description_train.shape[1])
    concat = Concatenate()([inputName, inputDesc, inputA])

    #x = Dropout(0.1)(concat)
    x = Dense(32, activation='relu')(concat)
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputName ,inputDesc, inputA], outputs=x)

    return model

In [ ]:
model = getModel_bow(new_tfidf_name_train, new_tfidf_description_train, categorical_validation_cvec)
model.summary() 

In [ ]:
plot_model(model)

In [ ]:
model.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
history = model.fit(x=[new_tfidf_name_train, new_tfidf_description_train, new_tfidf_categorical_train], y=y_train_cvec,
                    epochs=5,
                    verbose=True,
                    validation_data=([new_tfidf_name_validation, new_tfidf_description_validation, new_tfidf_categorical_validation], y_validation_cvec),
                    batch_size=512)
# 0.4771

In [ ]:
history2 = model.fit(x=[new_tfidf_name_train, new_tfidf_description_train], y=y_train_cvec,
                    epochs=5,
                    verbose=True,
                    validation_data=([new_tfidf_name_validation, new_tfidf_description_validation], y_validation_cvec),
                    batch_size=512)

In [ ]:
histdf = pd.DataFrame(history2.history)

In [ ]:
histdf.plot(y=["loss", "val_loss"])

tf-idf gensim

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.models import TfidfModel

In [ ]:
data["item_description"]

In [ ]:
doc_tokenized = [simple_preprocess(doc) for doc in data["item_description"].values]

In [ ]:
doc_tokenized[0]

In [ ]:
dictionary = corpora.Dictionary()

In [ ]:
BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokenized]

In [ ]:
# stampa le parole che compaiono in ogni descrizione con la loro frequenza di apparizione
for doc in BoW_corpus[:10]:
   print([[dictionary[id], freq] for id, freq in doc])

In [ ]:
tfidf = TfidfModel(BoW_corpus)

In [ ]:
for doc in tfidf[BoW_corpus]:
   print([[dictionary[id], np.around(freq)] for id, freq in doc])

In [ ]:
data["tfidf"]=tfidf[BoW_corpus]

In [ ]:
len(data.iloc[10]["tfidf"])

In [ ]:
train_tfidf, validation_tfidf = train_test_split(data[["item_condition_id", "shipping", "category_name_l",	"brand_name_l", "tfidf", "price"]], test_size=0.2, random_state=1000)

In [ ]:
train_tfidf_categorical=train_tfidf[["item_condition_id", "shipping", "category_name_l",	"brand_name_l"]]
validation_tfidf_categorical=validation_tfidf[["item_condition_id", "shipping", "category_name_l",	"brand_name_l"]]


train_tfidf_description=train_tfidf["tfidf"]
validation_tfidf_description=validation_tfidf["tfidf"]

y_train=train_tfidf["price"]
y_validation=validation_tfidf["price"]

In [ ]:
(validation_tfidf_description[0:10].todense)

In [ ]:
def getModel_bow(categorical_train, description_train):
    inputA = Input(categorical_train.shape[1])
    inputDesc = Input(description_train.shape[1])
    concat = Concatenate()([inputDesc, inputA])

    x = Dropout(0.1)(concat)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputDesc, inputA], outputs=x)

    return model

In [ ]:
model = getModel_bow(train_tfidf_categorical ,train_tfidf_description)
model.summary() 

In [ ]:
train_tfidf_description.shape

# Transformers

In [ ]:
train_trans_all = train
validation_trans = validation

In [ ]:
pret_model_trans = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
pret_model_trans.trainable = False

tokenizer_trans = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', use_fast=True)

In [ ]:
instances_per_batch = 200000

num_batches = int(train_trans_all.shape[0] / instances_per_batch)

trans_batch_starti = list(range(0, train_trans_all.shape[0], int(train_trans_all.shape[0] / num_batches)))[0:num_batches]
trans_batch_starti.append(train_trans_all.shape[0])

In [ ]:
trans_batches = [(trans_batch_starti[i], trans_batch_starti[i+1]) for i in range(0,len(trans_batch_starti)-1)]
[(i,b) for i,b in enumerate(trans_batches)]

In [ ]:
# divide in batch as we don't have enough memory to handle all at once
# 0 to 9
trans_batch_number = 0

trans_batch = trans_batches[trans_batch_number]
trans_batch

In [ ]:
print("Total train shape", train_trans_all.shape)
train_trans = train_trans_all[trans_batch[0]:trans_batch[1]]
print("Train batch shape", train_trans.shape)
print("Validation shape", validation_trans.shape)

In [ ]:
y_train_trans = train_trans["price"]
y_train_trans

In [ ]:
y_validation_trans = validation_trans["price"]
y_validation_trans

todo: use cleaned? or let bert handle everything?

In [ ]:
inputA_train_trans = train_trans[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
inputA_validation_trans = validation_trans[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
trans_name_tokenizer_maxlength = 50
trans_desc_tokenizer_maxlength = 50

In [ ]:
inputName_train_trans = tokenizer_trans(train_trans["name"].to_list(),
                                  return_tensors="tf",
                                  padding='max_length',
                                  truncation=True,
                                  max_length = trans_name_tokenizer_maxlength)
inputName_train_trans['input_ids'].shape

In [ ]:
inputName_validation_trans = tokenizer_trans(validation_trans["name"].to_list(),
                                  return_tensors="tf",
                                  padding='max_length',
                                  truncation=True,
                                  max_length = trans_name_tokenizer_maxlength)
inputName_validation_trans['input_ids'].shape

In [ ]:
inputName_shape_trans = (inputName_train_trans['input_ids'].shape[1],
                         inputName_train_trans['attention_mask'].shape[1])

inputName_shape_trans

In [ ]:
inputDesc_train_trans = tokenizer_trans(train_trans["item_description_clean"].to_list(),
                                  return_tensors="tf",
                                  padding='max_length',
                                  truncation=True,
                                  max_length=trans_desc_tokenizer_maxlength)
inputDesc_train_trans['input_ids'].shape

In [ ]:
inputDesc_validation_trans = tokenizer_trans(validation_trans["item_description_clean"].to_list(),
                                  return_tensors="tf",
                                  padding='max_length',
                                  truncation=True,
                                  max_length=trans_desc_tokenizer_maxlength)
inputDesc_validation_trans['input_ids'].shape

In [ ]:
inputDesc_shape_trans = (inputDesc_train_trans['input_ids'].shape[1],
                         inputDesc_train_trans['attention_mask'].shape[1])
inputDesc_shape_trans

In [ ]:
def getModel_trans():
    inputA = Input(shape=(4,))
    Ad = inputA    
    
    inputName_ids = Input(shape=(inputName_shape_trans[0],), dtype='int32')
    inputName_mask = Input(shape=(inputName_shape_trans[1],), dtype='int32')

    Np = pret_model_trans(inputName_ids, attention_mask=inputName_mask)[0]

    Nd = Bidirectional(LSTM(12, return_sequences=True, dropout=0.2))(Np)
    Nd =  GlobalMaxPool1D()(Nd)

    inputDesc_ids = Input(shape=(inputDesc_shape_trans[0],), dtype='int32')
    inputDesc_mask = Input(shape=(inputDesc_shape_trans[1],), dtype='int32')

    Dp = pret_model_trans(inputDesc_ids, attention_mask=inputDesc_mask)[0]

    Dd = Bidirectional(LSTM(16, return_sequences=True, dropout=0.2))(Dp)
    Dd = Bidirectional(LSTM(8, return_sequences=True, dropout=0.2))(Dd)
   
    Dd = GlobalMaxPool1D()(Dd)

    concat = Concatenate()([Ad, Nd, Dd])

    x = Dropout(0.2)(concat)    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[
                          inputA,
                          inputName_ids,
                          inputName_mask,
                          inputDesc_ids,
                          inputDesc_mask
                          ], outputs=x)
    
    return model
    

In [ ]:
model_trans = getModel_trans()

In [ ]:
model_trans.summary()

In [ ]:
plot_model(model_trans)

In [ ]:
model_trans.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
history_trans = model_trans.fit(x=[inputA_train_trans,
                   inputName_train_trans['input_ids'],
                   inputName_train_trans['attention_mask'],
                   inputDesc_train_trans['input_ids'],
                   inputDesc_train_trans['attention_mask']
                  ],
                   y=y_train_trans,
                    epochs=10,
                    verbose=True,
                    validation_data=([
                                      inputA_validation_trans,
                                      inputName_validation_trans['input_ids'],
                                      inputName_validation_trans['attention_mask'],
                                      inputDesc_validation_trans['input_ids'],
                                      inputDesc_validation_trans['attention_mask'],
                                      ], 
                                     y_validation_trans),
                    batch_size=512)

453/453 [==============================] - 1733s 4s/step - loss: 0.5675 - mse: 1256.7464 - mae: 13.4240 - root_mean_squared_error: 35.4326 - mean_squared_logarithmic_error: 0.3226 - root_mean_squared_logarithmic_error: 0.5675 - val_loss: 0.5444 - val_mse: 1157.3416 - val_mae: 12.7427 - val_root_mean_squared_error: 34.0197 - val_mean_squared_logarithmic_error: 0.2970 - val_root_mean_squared_logarithmic_error: 0.5445